In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 100

In [2]:
def open_df(file): 
    #file_path = '/mnt/e/Box Sync/Baylor/Projects/Xylitol/Data From Katie 12 Jan 2022/PPAX FINAL DATABASE.xlsx' # unhash for unix for windows subsystem
    file_path = "E:\Box Sync\Baylor\Projects\Xylitol\Data From Katie 12 Jan 2022\\"
    df = pd.read_csv(file_path + file, index_col="Site_ID")
    df = df.apply(pd.to_numeric, errors='coerce')
    return file_path, df

In [3]:
def save_df(df, file_path): 
    savename = file_path + "Cannon_composite.csv"
    df.to_csv(savename)

In [4]:
def get_visit (filename): 
    return filename.rsplit('_', 1)[1][:2]

In [5]:
def attachment_loss_binary (out_data): 
    files = ('attachment_loss_V0.csv', 'attachment_loss_V1.csv')
    for file in files:
        visit = get_visit (file)
        file_path, df = open_df (file)    
        df.replace('.',  np.nan, inplace=True)

        out_data['Attachment_loss_' + visit] = df.apply(lambda row: None if row.isnull().all() 
                                         else any(row.isin([1, 2, 3, 4])), axis=1)
    return (out_data)       

In [6]:
def pockets_binary (out_data): 
    files = ('pocket_depth_V0.csv', 'pocket_depth_V1.csv')
    for file in files: 
        visit = get_visit (file)
        file_path, df = open_df (file)    
        df.replace('.',  np.nan, inplace=True)

        out_data['Pockets_' + visit] = df.apply(lambda row: None if row.isnull().all() 
                                         else any(row.isin([1, 2])), axis=1)
    return out_data

In [7]:
def Cannon_composite (out_data): 

    def calculate_Cannon (row, visit): 
        if pd.isna(row['Attachment_loss'+ visit]) or pd.isna(row['Pockets' + visit]):
            return None
        else:
            return row['Attachment_loss' + visit] or row['Pockets'+ visit]
        
    
    def calculate_periodontitis(row, visit):
        if pd.isna(row['Attachment_loss'+ visit]) or pd.isna(row['Pockets' + visit]):
            return None
        else:
            return row['Attachment_loss' + visit] and row['Pockets'+ visit]
    
    # Apply the function to each row
    visits = ('_V0', '_V1')
    for visit in visits: 
        out_data['Periodontitis' + visit] = out_data.apply(calculate_periodontitis, axis=1, args=(visit,))
        out_data['Cannon_comp' + visit] = out_data.apply(calculate_Cannon, axis=1, args=(visit,))
    
    display(out_data)
    true_counts = out_data.sum()
    print (true_counts)
    out_data.replace({True: 1, False: 0, None: ''}, inplace=True)
    return out_data

In [8]:
filepath , out_data = open_df('attachment_loss_V0.csv')
out_data = out_data.iloc[:, []]
out_data = attachment_loss_binary(out_data)
out_data = pockets_binary (out_data)       
out_data = Cannon_composite (out_data)
save_df (out_data, filepath)

,Attachment_loss_V0,Attachment_loss_V1,Pockets_V0,Pockets_V1,Periodontitis_V0,Cannon_comp_V0,Periodontitis_V1,Cannon_comp_V1
Site_ID,,,,,,,,
A_1108,False,None,False,None,False,False,None,None
A_1109,None,None,None,None,None,None,None,None
A_1110,None,None,None,None,None,None,None,None
A_1111,False,None,False,None,False,False,None,None
A_1112,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
H_1032,False,False,False,False,False,False,False,False
H_1033,None,None,None,None,None,None,None,None
H_1034,None,None,None,None,None,None,None,None


Attachment_loss_V0     92
Attachment_loss_V1     17
Pockets_V0            207
Pockets_V1             29
Periodontitis_V0       62
Cannon_comp_V0        237
Periodontitis_V1        8
Cannon_comp_V1         38
dtype: object
